In [10]:
%load_ext autoreload
%autoreload 1

import numpy as np
import lasagne as L
from squad_load import get_glove_train_embs, get_squad_train_voc, load_squad_train, get_squad_train_chars
from itertools import chain

%aimport QANet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
%%time
squad_path = '/pio/data/data/squad/'
glove_path = '/pio/data/data/glove_vec/6B/glove/'

data = np.load(squad_path + 'train_with_glove_vocab.pkl')
data_char = np.load(squad_path + 'train_char_ascii.pkl')
train_bin_feats = np.load(squad_path + 'train_bin_feats.pkl')
glove_embs = np.load(glove_path + 'glove.6B.300d.npy')
voc_size = glove_embs.shape[0]
alphabet_size = 128

# Some answers get broken in the process of tokenization, because some answer words are not properly split.
def filter_broken_answers(data, data_char, train_bin_feats):
    data_new = []
    data_char_new = []
    train_bin_feats_new = []
    for i in xrange(len(data)):
        if data[i][0]:
            data_new.append(data[i])
            data_char_new.append(data_char[i])
            train_bin_feats_new.append(train_bin_feats[i])
    return data_new, data_char_new, train_bin_feats_new

def trim_data(data, data_char, train_bin_feats, trim=300):
    data_new = []
    data_char_new = []
    train_bin_feats_new = []
    for i in xrange(len(data)):
        if len(data[i][2]) > trim:
            if data[i][0][0][-1] < trim:
                data_new.append(data[i][:2] + [data[i][2][:trim]])
                data_char_new.append([data_char[i][0], data_char[i][1][:trim]])
                train_bin_feats_new.append(train_bin_feats[i][:trim])
        else:
            data_new.append(data[i])
            data_char_new.append(data_char[i])
            train_bin_feats_new.append(train_bin_feats[i])
    return data_new, data_char_new, train_bin_feats_new

print 'Examples total:', len(data), len(data_char), len(train_bin_feats)

data, data_char, train_bin_feats = filter_broken_answers(data, data_char, train_bin_feats)
data, data_char, train_bin_feats = trim_data(data, data_char, train_bin_feats)

print 'Working examples:', len(data), len(data_char), len(train_bin_feats)

data = (data, data_char, train_bin_feats)

Examples total: 87599 87599 87599
Working examples: 86355 86355 86355
CPU times: user 1min 14s, sys: 326 ms, total: 1min 14s
Wall time: 1min 14s


In [12]:
%%time
data_dev = np.load(squad_path + 'dev_with_glove_vocab.pkl')
data_dev_char = np.load(squad_path + 'dev_char_ascii.pkl')
dev_bin_feats = np.load(squad_path + 'dev_bin_feats.pkl')

CPU times: user 9.77 s, sys: 19.5 ms, total: 9.79 s
Wall time: 9.8 s


In [108]:
data_dev_all = data_dev, data_dev_char, dev_bin_feats

## QANet

In [147]:
%%time
qa_net = QANet.QANet(voc_size=voc_size,
                     alphabet_size=alphabet_size,
                     emb_size=300,
                     emb_char_size=20,
                     num_emb_char_filters=200,
                     rec_size=300,
                     emb_init=glove_embs,
                     train_inds=[])

Building the model...
Compiling theano functions:
    train_fn...
    get_start_probs_fn...
    get_end_probs_fn...
Done
CPU times: user 1min 11s, sys: 1.33 s, total: 1min 12s
Wall time: 1min 12s


In [150]:
qa_net.learning_rate

0.0005

In [15]:
# qa_net.load_params('trained_models/glove_vocab/charemb_all_fixed_ep3.npz', glove_embs)

In [149]:
# z char-embeddings

# dane są przycięte do długości 300 (jeśli odpowiedź się nie mieści, to pytanie jest usuwane z danych)
# przycięto około 1400 próbek, usunięto 119

# słownik glove, unk to średnie słowo, nie trenujemy zanurzeń słów, char-embeddings ma losowy init

qa_net.train_one_epoch(data, 15, log_interval=5)

Done 5 batches in 5.63s	training loss:	9.283470
Done 10 batches in 9.76s	training loss:	8.741427
Done 15 batches in 14.02s	training loss:	8.487012
Done 20 batches in 18.36s	training loss:	8.358544
Calculating validation f1...
Done 1000 examples
Done 2000 examples
Done 3000 examples
Done 4000 examples
Done 5000 examples
Done 6000 examples
Done 7000 examples
Done 8000 examples
Done 9000 examples
Done 10000 examples
Predictions done
F1:  8.27234666418
Done 25 batches in 283.49s	training loss:	8.171664
Done 30 batches in 287.56s	training loss:	7.942222
Done 35 batches in 291.60s	training loss:	7.719355
Done 40 batches in 295.70s	training loss:	7.661161
Calculating validation f1...
Done 1000 examples
Done 2000 examples
Done 3000 examples
Done 4000 examples
Done 5000 examples
Done 6000 examples
Done 7000 examples
Done 8000 examples
Done 9000 examples
Done 10000 examples
Predictions done
F1:  8.57612258945
Done 45 batches in 556.93s	training loss:	7.599194
Done 50 batches in 560.72s	training 

KeyboardInterrupt: 

In [51]:
qa_net.save_params('trained_models/glove_vocab/charemb_all_fixed_ep4')

### QANet tests

In [ ]:
'''
charemb all_fixed 2ep {"f1": 63.93851222573222, "exact_match": 53.33964049195837} 
charemb all_fixed 3ep {"f1": 64.65644340794375, "exact_match": 54.71144749290445}
charemb all_fixed 3ep with premadeBin {"f1": 65.03551561149246, "exact_match": 55.052034058656574}

'''

In [16]:
def predict_spans(data, beam=10, batch_size=10, premade_bin_feats=True):
    num_examples = len(data[0])
    
    start_probs = qa_net.get_start_probs(data, batch_size, premade_bin_feats=premade_bin_feats)
    best_starts = start_probs.argpartition(-beam, axis=1)[:, -beam:].astype(np.int32)
    
    scores = start_probs[np.arange(num_examples)[:, np.newaxis], best_starts]
    scores = np.tile(scores[:, np.newaxis], (beam, 1)).transpose(0, 2, 1)
    
    best_ends_all = []
    for i in xrange(beam):
        end_probs = qa_net.get_end_probs(data, best_starts[:, i], batch_size, 
                                         premade_bin_feats=premade_bin_feats)
        best_ends = end_probs.argpartition(-beam, axis=1)[:, -beam:]
        scores[:, i, :] *= end_probs[np.arange(num_examples)[:, np.newaxis], best_ends]
        best_ends_all.append(best_ends)
        
    best_ends_all = np.hstack(best_ends_all)
        
    scores = scores.reshape(num_examples, beam**2)
    best_spans = scores.argmax(axis=1)
    starts = [i / beam for i in best_spans]
    
    starts = best_starts[np.arange(num_examples), starts]
    ends = best_ends_all[np.arange(num_examples), best_spans]
    
    return starts, ends

In [18]:
%%time
predicted_spans = []
batch_size = 10

idx = 0
while idx < len(data_dev):
    data_dev_batch = [data_dev[idx:idx + batch_size], 
                      data_dev_char[idx:idx + batch_size], 
                      dev_bin_feats[idx:idx + batch_size]]
    spans = predict_spans(data_dev_batch, beam=1)
    predicted_spans.append(np.vstack(spans))
    idx += batch_size
    if not idx % 1000:
        print idx, 'examples'
        
print 'Predictions done'
    
predicted_spans = np.hstack(predicted_spans).T

1000 examples
2000 examples
3000 examples
4000 examples
5000 examples
6000 examples
7000 examples
8000 examples
9000 examples
10000 examples
Predictions done
CPU times: user 13min 42s, sys: 36min 23s, total: 50min 5s
Wall time: 4min 14s


In [19]:
np.savez('evaluate/glove_vocab/pred_glove_premadeBin_charemb_all_fixed_ep3.npz', predicted_spans)